# Day 16: The Floor Will Be Lava

## Part 1

With the beam of light completely focused somewhere, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.

Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the beam of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.

Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing empty space (`.`), mirrors (`/` and `\`), and splitters (`|` and `-`).

The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into heat to melt the rock in the cavern.

You note the layout of the contraption (your puzzle input). For example:

```
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
```

The beam enters in the top-left corner from the left and heading to the right. Then, its behavior depends on what it encounters as it moves:

- If the beam encounters empty space (`.`), it continues in the same direction.
- If the beam encounters a mirror (`/` or `\`), the beam is reflected 90 degrees depending on the angle of the mirror. For instance, a rightward-moving beam that encounters a `/` mirror would `-` continue upward in the mirror's column, while a rightward-moving beam that encounters a `\` mirror would continue downward from the mirror's column.
- If the beam encounters the pointy end of a splitter (`|` or `-`), the beam passes through the splitter as if the splitter were empty space. For instance, a rightward-moving beam that `-` encounters a `-` splitter would continue in the same direction.
- If the beam encounters the flat side of a splitter (`|` or `-`), the beam is split into two beams going in each of the two directions the splitter's pointy ends are pointing. For instance, a rightward-moving beam that encounters a `|` splitter would split into two beams: one that continues upward from the splitter's column and one that continues downward from the splitter's column.

Beams do not interact with other beams; a tile can have many beams passing through it at the same time. A tile is energized if that tile has at least one beam pass through it, reflect in it, or split in it.

In the above example, here is how the beam of light bounces around the contraption:

```
>|<<<\....
|v-.\^....
.v...|->>>
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
<->-/vv|..
.|<<<2-|.\
.v//.|.v..
```

Beams are only shown on empty tiles; arrows indicate the direction of the beams. If a tile contains beams moving in multiple directions, the number of distinct directions is shown instead. Here is the same diagram but instead only showing whether a tile is energized (`#`) or not (`.`):

```
######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..
```

Ultimately, in this example, `46` tiles become energized.

The light isn't energizing enough tiles to produce lava; to debug the contraption, you need to start by analyzing the current situation. With the beam starting in the top-left heading right, how many tiles end up being energized?



In [47]:
import numpy as np


test_data_raw = (
r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|...."""
)


def parse_data(data):
    return np.array([list(line) for line in data.splitlines()])


def print_data(data):
    for line in data:
        print("".join(map(str, line)))


test_data = parse_data(test_data_raw)
print_data(test_data)

.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....


In [48]:
def get_new_position(position, direction):
    return position[0] + direction[0], position[1] + direction[1]


def perform_beams_step(mirrors, beams, traces, directions):
    new_beams = []
    for beam in beams:
        position, direction = beam
        if (
            0 <= position[0] < mirrors.shape[0] 
            and 0 <= position[1] < mirrors.shape[1]
        ):
            traces[*position] = True
            mirror = mirrors[*position]
            if not directions[*position]:
                directions[*position] = set()
            if direction not in directions[*position]:
                directions[*position].add(direction)
                if mirror in "\\/":
                    a = 1 if mirror == "\\" else -1
                    direction = direction[1] * a, direction[0] * a
                    new_beams.append((get_new_position(position, direction), direction))
                elif mirror == "|" and direction in {(0, 1), (0, -1)}:
                    for new_direction in [(-1, 0), (1, 0)]:
                        new_beams.append((get_new_position(position, new_direction), new_direction))
                elif mirror == "-" and direction in {(-1, 0), (1, 0)}:
                    for new_direction in [(0, -1), (0, 1)]:
                        new_beams.append((get_new_position(position, new_direction), new_direction))
                else:
                    new_beams.append((get_new_position(position, direction), direction))
    return new_beams


def get_energized_count(mirrors, start_beam_position=(0, 0), start_beam_direction=(0, 1)):
    traces = np.zeros(mirrors.shape, dtype=bool)
    directions = np.full(mirrors.shape, None)
    beams = [(start_beam_position, start_beam_direction)]
    while len(beams) > 0:
        beams = perform_beams_step(mirrors, beams, traces, directions)
    return traces.sum()


def part1(data):
    return get_energized_count(data)


part1(test_data)

46

In [49]:
with open("input.txt") as f:
    data = parse_data(f.read())

print_data(data[:5, :5])

\....
.....
....-
....\
..|.|


In [50]:
part1(data)

8112

## Part 2

As you try to work out what might be wrong, the reindeer tugs on your shirt and leads you to a nearby control panel. There, a collection of buttons lets you align the contraption so that the beam enters from any edge tile and heading away from that edge. (You can choose either of two directions for the beam if it starts on a corner; for instance, if the beam starts in the bottom-right corner, it can start heading either left or upward.)

So, the beam could start on any tile in the top row (heading downward), any tile in the bottom row (heading upward), any tile in the leftmost column (heading right), or any tile in the rightmost column (heading left). To produce lava, you need to find the configuration that energizes as many tiles as possible.

In the above example, this can be achieved by starting the beam in the fourth tile from the left in the top row:

```
.|<2<\....
|v-v\^....
.v.v.|->>>
.v.v.v^.|.
.v.v.v^...
.v.v.v^..\
.v.v/2\\..
<-2-/vv|..
.|<<<2-|.\
.v//.|.v..
```

Using this configuration, `51` tiles are energized:

```
.#####....
.#.#.#....
.#.#.#####
.#.#.##...
.#.#.##...
.#.#.##...
.#.#####..
########..
.#######..
.#...#.#..
```

Find the initial beam configuration that energizes the largest number of tiles; how many tiles are energized in that configuration?

In [51]:
def part2(data):
    result = 0
    for start_position in [
        *[(0, i) for i in range(data.shape[1])],
        *[(data.shape[0], i) for i in range(data.shape[1])],
        *[(i, 0) for i in range(data.shape[0])],
        *[(i, data.shape[1]) for i in range(data.shape[0])]
    ]:
        for direction in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            result = max(result, get_energized_count(data, start_position, direction))
    return result


part2(test_data)

51

In [52]:
part2(data)

8314